In [1]:
from matplotlib import pyplot as plt
from numpy.linalg import svd
import numpy as np

from skimage import data
from skimage.color import rgb2gray
from skimage import img_as_ubyte,img_as_float

from sklearn.decomposition import PCA, TruncatedSVD

%matplotlib inline

In [65]:
train = np.loadtxt(r'../data/MNIST_train_small.csv', delimiter=',')[:,1:]

def compress_image(k):
    image= train[1].reshape(28, 28)
    original_shape = image.shape
    reconst_img,s = compress_svd(image, k)
    fig,axes = plt.subplots(1, 2, figsize=(8,5))
    axes[0].plot(s)
    compression_ratio =100.0* (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1])
    axes[1].set_title("compression ratio={:.2f}".format(compression_ratio)+"%")
    axes[1].imshow(reconst_img,cmap='gray')
    axes[1].axis('off')
    fig.tight_layout()

def compress_svd(image, k):
    """
    SVD decomposition, trucating image using k singular values
    returns:
        reconstructed matrix
        matrix sigma with singular values
    """
    U, sigma, V = svd(image, full_matrices=False)
    reconst_matrix = np.dot(U[:,:k], np.dot(np.diag(sigma[:k]), V[:k,:]))
    return reconst_matrix, sigma

In [66]:
interact(compress_image, k=(1,20));

interactive(children=(IntSlider(value=10, description='k', max=20, min=1), Output()), _dom_classes=('widget-in…

In [50]:
for k in range(2, 11):
    imgs_trunc = []
    for img in train:
        img_r = img.reshape(28, 28)
        img_trunc, _ = compress_svd(img.reshape(28, 28), 2)
        imgs_trunc.append(img_trunc)
    np.savetxt(f'../data/train_{k}.csv', np.array(imgs_trunc).reshape((3,-1)), delimiter=',')